In [1]:
import pandas as pd
from rapidfuzz import process, fuzz

In [4]:
# Load Instacart products
products_df = pd.read_csv('../data/instacart-market-basket-analysis/products.csv')
instacart_products = products_df['product_name'].str.lower().str.strip()

In [9]:
# path = '../data/groceries/groceries.csv'
path = '../data/store-data/store_data.csv'
# path = '../data/retail-transactions/Retail_Transactions_Dataset.csv'

In [10]:
# Load products from another dataset (e.g., transactions)
with open(path, 'r') as file:
    transactions = [line.strip().lower() for line in file]

# Normalize names
def normalize(name):
    return ''.join(e for e in name if e.isalnum() or e.isspace()).strip()

instacart_products = instacart_products.apply(normalize)
transactions = [normalize(item) for item in transactions]

# Apply fuzzy matching
threshold = 80  # Similarity threshold
matches = []

for product in instacart_products:
    best_match = process.extractOne(product, transactions, scorer=fuzz.token_sort_ratio)
    if best_match and best_match[1] >= threshold:
        matches.append((product, best_match[0], best_match[1]))  # (instacart_product, matched_product, score)

# Convert matches to DataFrame
matches_df = pd.DataFrame(matches, columns=['Instacart Product', 'Matched Product', 'Similarity Score'])

# Save to CSV
matches_df.to_csv('product_matches.csv', index=False)

print(matches_df.head())


         Instacart Product       Matched Product  Similarity Score
0                     soda                  soda        100.000000
1   shaved parmesan cheese       parmesan cheese         81.081081
2                   turkey                turkey        100.000000
3  whole wheat spanakopita     whole wheat pasta         80.000000
4   dark chocolate raisins  extra dark chocolate         80.952381


Conclusion: Dataset products intertwine enough, no need for clustering